# **Handling video files in Python**

In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

# **Reading the video, extracting frames from it and save them as images**

In [2]:
count = 0
videoFile = "/content/Cristiano Ronaldo Best Emotional And Happy Moments _Portugal _Into Your Arms Short Video #shorts#CR7.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


## **Label a images for training the model**

In [3]:
data = pd.read_csv('/content/mapping - mapping.csv')     # reading the csv file
data.head()      # printing first five rows of the file

,Image_ID,Class
0,frame0.jpg,0
1,frame1.jpg,0
2,frame2.jpg,0
3,frame3.jpg,0
4,frame4.jpg,0


In [4]:
X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

# **Onehot Encoding**

Since there are three classes, we will one hot encode them using the to_categorical() function of keras.utils.

In [5]:
y = data.Class
dummy_y = np_utils.to_categorical(y)  

# **Using VGG16 MOdel**

We will be using a VGG16 pretrained model which takes an input image of shape (224 X 224 X 3). Since our images are in a different size, we need to reshape all of them. We will use the resize() function of skimage.transform to do this.

In [6]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)    
    image.append(a)
X = np.array(image)

# **Preprocessing Of reshaped Images**

In [7]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)      # preprocessing the input data

# **Train Test Splitting**

We also need a validation set to check the performance of the model on unseen images. We will make use of the train_test_split() function of the sklearn.model_selection module to randomly divide images into training and validation set.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)  

# **Bulding,Compiling,Training the Model**

In [9]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

58889256/58889256 [==============================] - 3s 0us/step


In [11]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

1/1 [==============================] - 6s 6s/step


((25, 7, 7, 512), (11, 7, 7, 512))

In [12]:
X_train = X_train.reshape(25, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(11, 7*7*512)

In [13]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [14]:
# i. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(3, activation='softmax'))    # output layer

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________


In [16]:
# ii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# iii. Training the model
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
1/1 [==============================] - 1s 917ms/step - loss: 1.0363 - accuracy: 0.6000 - val_loss: 0.5535 - val_accuracy: 0.7273
Epoch 2/100
1/1 [==============================] - 0s 188ms/step - loss: 0.4187 - accuracy: 0.8800 - val_loss: 0.4243 - val_accuracy: 0.8182
Epoch 3/100
1/1 [==============================] - 0s 207ms/step - loss: 0.1998 - accuracy: 0.8800 - val_loss: 0.3756 - val_accuracy: 0.8182
Epoch 4/100
1/1 [==============================] - 0s 188ms/step - loss: 0.0723 - accuracy: 1.0000 - val_loss: 0.3605 - val_accuracy: 0.8182
Epoch 5/100
1/1 [==============================] - 0s 199ms/step - loss: 0.0246 - accuracy: 1.0000 - val_loss: 0.3633 - val_accuracy: 0.8182
Epoch 6/100
1/1 [==============================] - 0s 196ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.3752 - val_accuracy: 0.8182
Epoch 7/100
1/1 [==============================] - 0s 215ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.3913 - val_accuracy: 0.8182
Epoch 8/100
1

In [18]:
scores = model.evaluate(train, y_train)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1/1 [==============================] - 0s 59ms/step - loss: 5.4279e-05 - accuracy: 1.0000
accuracy: 100.00%


# **Model Evaluation Using Test data**

In [19]:
count = 0
videoFile = "/content/Crying rono.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [20]:
data1 = pd.read_csv('/content/test - test1.csv')     # reading the csv file
data1.head()      # printing first five rows of the file

,Image_ID
0,test0.jpg
1,test1.jpg
2,test2.jpg
3,test3.jpg
4,test4.jpg


In [21]:
data1.shape

(30, 1)